<a href="https://colab.research.google.com/github/NHendra/vits/blob/main/Sovits_Infer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!nvidia-smi


Sat Apr  1 16:50:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   75C    P0    34W /  70W |   3599MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [26]:
!git clone https://github.com/svc-develop-team/so-vits-svc
%cd so-vits-svc

Cloning into 'so-vits-svc'...
remote: Enumerating objects: 805, done.
remote: Counting objects: 100% (170/170), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 805 (delta 157), reused 155 (delta 154), pack-reused 635
Receiving objects: 100% (805/805), 8.22 MiB | 27.77 MiB/s, done.
Resolving deltas: 100% (372/372), done.
/content/so-vits-svc/so-vits-svc


In [27]:
!pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
!pip install pyworld --no-cache-dir
!pip install $(grep -ivE "pyworld" requirements.txt)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
#@title Download models

!wget -nc -P logs/alice https://huggingface.co/spaces/sayashi/Sovits-Umamusume/resolve/main/models/alice/alice.pth
!wget -nc -P logs/alice https://huggingface.co/spaces/sayashi/Sovits-Umamusume/resolve/main/models/alice/config.json

!wget -nc -P hubert/ https://huggingface.co/spaces/sayashi/Sovits-Umamusume/resolve/main/hubert/checkpoint_best_legacy_500.pt

--2023-04-01 16:50:51--  https://huggingface.co/spaces/sayashi/Sovits-Umamusume/resolve/main/models/alice/alice.pth
Resolving huggingface.co (huggingface.co)... 52.2.178.255, 35.173.225.216, 34.206.0.154, ...
Connecting to huggingface.co (huggingface.co)|52.2.178.255|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: /spaces/sayashi/sovits-models/resolve/main/models/alice/alice.pth [following]
--2023-04-01 16:50:51--  https://huggingface.co/spaces/sayashi/sovits-models/resolve/main/models/alice/alice.pth
Reusing existing connection to huggingface.co:443.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/88/2d/882dd332f5381f6815f7b59356765a8e114da272f764e794de476bd69ff7be7e/6d1bef76e26beeadcae5f716cc0b60abb2aac4aae1316cac709cc439726cf533?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27alice.pth%3B+filename%3D%22alice.pth%22%3B&Expires=1680627052&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cm

In [30]:
#@title  Upload Audio
%cd raw
from google.colab import files
uploaded = files.upload()
%cd ..

/content/so-vits-svc/so-vits-svc/raw


Saving download.flac to download.flac
/content/so-vits-svc/so-vits-svc


In [32]:
#@title  Infer
import io
import logging
import time
from pathlib import Path
import IPython.display as ipd
import librosa
import matplotlib.pyplot as plt
import numpy as np
import soundfile
import torch
from inference import infer_tool
from inference import slicer
from inference.infer_tool import Svc

logging.getLogger('numba').setLevel(logging.WARNING)
chunks_dict = infer_tool.read_temp("inference/chunks_temp.json")

In [34]:
model_name = "alice" #@param ["teio", "tannhauser", "alice", "goldship", "rudolf"]
model_path = f"logs/{model_name}/{model_name}.pth"
config_path = f"logs/{model_name}/config.json"
input_wav = "download.flac" #@param {type:"string"}
vc_transform = 1 #@param {type:"number"}
auto_f0 = True #@param {type:"boolean"}
try:
  del svc_model.net_g_ms
  del svc_model
  torch.cuda.empty_cache()
  torch.cuda.empty_cache()
except:
  pass
svc_model = Svc(model_path, config_path)
infer_tool.mkdir(["raw", "results"])
clean_names = [input_wav]
trans = [vc_transform]
slice_db = -40
wav_format = "flac"
auto_predict_f0 = auto_f0
pad_seconds = 0.5
spk = model_name

infer_tool.fill_a_to_b(trans, clean_names)
for clean_name, tran in zip(clean_names, trans):
  raw_audio_path = f"raw/{clean_name}"
  if "." not in raw_audio_path:
      raw_audio_path += ".wav"
  infer_tool.format_wav(raw_audio_path)
  wav_path = Path(raw_audio_path).with_suffix('.wav')
  chunks = slicer.cut(wav_path, db_thresh=slice_db)
  audio_data, audio_sr = slicer.chunks2audio(wav_path, chunks)

  audio = []
  for (slice_tag, data) in audio_data:
      print(f'#=====segment start, {round(len(data) / audio_sr, 3)}s======')
      # padd
      pad_len = int(audio_sr * pad_seconds)
      data = np.concatenate([np.zeros([pad_len]), data, np.zeros([pad_len])])
      length = int(np.ceil(len(data) / audio_sr * svc_model.target_sample))
      raw_path = io.BytesIO()
      soundfile.write(raw_path, data, audio_sr, format="wav")
      raw_path.seek(0)
      if slice_tag:
          print('jump empty segment')
          _audio = np.zeros(length)
      else:
          out_audio, out_sr = svc_model.infer(
              spk, tran, raw_path,
              auto_predict_f0=auto_predict_f0
          )
          _audio = out_audio.cpu().numpy()

      pad_len = int(svc_model.target_sample * pad_seconds)
      _audio = _audio[pad_len:-pad_len]
      audio.extend(list(_audio))
  key = "auto" if auto_predict_f0 else f"{tran}key"
  res_path = f'./results/{clean_name}_{key}_{spk}.{wav_format}'
  soundfile.write(res_path, audio, svc_model.target_sample, format=wav_format)
  ipd.display(ipd.Audio(res_path, rate=44100, normalize=False))

load model(s) from hubert/checkpoint_best_legacy_500.pt
load 
#=====segment start, 9.26s======
vits use time:0.21367883682250977
#=====segment start, 5.38s======
vits use time:0.12193107604980469
#=====segment start, 14.68s======
vits use time:0.22472023963928223
#=====segment start, 15.02s======
vits use time:0.22625160217285156
#=====segment start, 0.007s======
jump empty segment
